In [4]:
import requests, csv, json, urllib
import pandas as pd
import time
!pip install fake-useragent
from fake_useragent import UserAgent
from datetime import datetime

BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata/"
START_DATE = '2023-01-01'
END_DATE = '2025-11-19'
ua = UserAgent()

headers = {
   'User-Agent': ua.random,
   }

r = requests.get(BASE_URL + START_DATE, headers = headers)
data = r.json()

fng_data = pd.read_csv('fear-greed-2011-2023.csv', usecols=['Date', 'Fear Greed'])

# Correct automatic date parsing
fng_data['Date'] = pd.to_datetime(fng_data['Date'], format='mixed')

fng_data.set_index('Date', inplace=True)

# Expand index to full date range
all_dates = pd.date_range(fng_data.index[0], pd.to_datetime(END_DATE), freq='D')
missing_dates = []

for date in all_dates:
    if date not in fng_data.index:
        missing_dates.append(date)
        fng_data.loc[date] = [0]

fng_data.sort_index(inplace=True)

# Insert CNN F&G data
for point in data['fear_and_greed_historical']['data']:
    ts = pd.to_datetime(point['x'], unit='ms')
    val = int(point['y'])
    fng_data.at[ts, 'Fear Greed'] = val

fng_data.to_csv('all_fng_csv.csv')

In [5]:
import pandas as pd

# Load and normalize timestamps
df = pd.read_csv("all_fng_csv.csv", parse_dates=["Date"])
df["Date"] = df["Date"].dt.normalize()     # remove hours
df = df.groupby("Date").first()            # drop duplicates (keeps first)

# Weekend removal: Saturday=5, Sunday=6
df["weekday"] = df.index.weekday

# Drop weekends only if Fear Greed == 0
df = df[~((df["weekday"] >= 5) & (df["Fear Greed"] == 0))]

df = df.drop(columns=["weekday"])

# Save
df.to_csv("all_fng_csv_cleaned.csv")
